<a href="https://colab.research.google.com/github/VinKKAP/Data-Analysis-with-LLM/blob/main/TryTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

# Upgrade pip und installiere die benötigten Pakete
%pip install --upgrade pip
%pip install --upgrade torch transformers
%pip install "accelerate>=0.26.0"

# Überprüfen, ob CUDA verfügbar ist, und den Namen des CUDA-Geräts abrufen
print("CUDA verfügbar:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA-Gerät:", torch.cuda.get_device_name(0))
else:
    print("CUDA wird nicht unterstützt.")


CUDA verfügbar: True
CUDA-Gerät: Tesla T4


GPU ist verbunden für die berechnung. Aber die Instruction für das Modell scheitert daran das dass jeweilige Modell nicht richtig im Code aus dem Buch funktioniert.

In [17]:
import argparse
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import re
import time

# Check and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Lade das T5-Modell und den Tokenizer
model_name = "t5-base"  # Changed to a model suitable for text2text-generation
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Erstelle eine Pipeline für Textgenerierung oder Textklassifikation
nlp = pipeline('text2text-generation', model=model, tokenizer=tokenizer,
                device=device,
                truncation=True)

def create_prompt(question):  #1
    """ Generate prompt to translate question into SQL query.

    Args:
        question: question about data in natural language.

    Returns:
        prompt for question translation.
    """
    parts = []
    parts += ['Database:']
    parts += ['create table games(rank int, name text, platform text,']
    parts += ['year int, genre text, publisher text, americasales numeric,']
    parts += ['eusales numeric, japansales numeric, othersales numeric,']
    parts += ['globalsales numeric);']
    parts += ['Translate this question into SQL query:']
    parts += [question]
    return '\textbackslash{}n'.join(parts)


def call_llm(prompt):  #2
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    for nr_retries in range(1, 4):
        try:
            response = nlp(prompt, max_length=150)
            return response[0]['generated_text']
        except Exception as e:
            print(f"Error: {e}")
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query model!')


if __name__ == '__main__':  #3

    parser = argparse.ArgumentParser()
    parser.add_argument('question', type=str, help='A question about games')
    args = parser.parse_args(['how many games are there?'])

    prompt = create_prompt(args.question)
    answer = call_llm(prompt) #4
    query = re.findall('```sql(.*)```', answer, re.DOTALL)
    if query:
        print(f'SQL: {query[0]}')
    else:
        print("No SQL query found in the response.")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


No SQL query found in the response.
